In [ ]:
import os
import pywt
import pathlib
import pyedflib
import numpy as np
import pandas as pd
from pyentrp import entropy
from scipy import signal, stats, integrate
from sklearn import metrics, preprocessing, decomposition

## EDF Files Manipulation

### Read an edf file

In [ ]:
'''
https://stackoverflow.com/questions/48784257/convert-eye-tracking-edf-file-to-asc-csv-format
https://pyedflib.readthedocs.io/en/latest/
https://www.edfplus.info/specs/edf.html
'''
def readEdfFile(pathToFile):
    f = pyedflib.EdfReader(pathToFile)
    n = f.signals_in_file
    signal_labels = f.getSignalLabels()
    sigbufs = np.zeros((f.getNSamples()[0],n))
    for i in np.arange(n):
        sigbufs[:,i] = f.readSignal(i)
    f._close()
    del i,f,n,signal_labels
    return sigbufs

### Read an edf seizures file

In [ ]:
'''
https://www.mathworks.com/matlabcentral/answers/225716-how-i-can-read-chb01_03-edf-seizures-file-from-chb-mit-database-in-matlab-as-i-am-using-this-file-f
Returns start time and length of the seizure
'''
def get_seizure_period(file_location):
    bytes_array = []
    for b in pathlib.Path(file_location).read_bytes(): bytes_array.append(b)
    return int(str(bin(bytes_array[38]))[2:]+str(bin(bytes_array[41]))[2:],2), bytes_array[49]

## List of numpy array, each position contains a patient's array of data

In [ ]:
def read_and_store_data (dataset_folder, sample_rate) :
    path = os.path.join(os.getcwd(), dataset_folder)
    os.chdir(dataset_folder)
    
    patients = [d for d in os.listdir() if os.path.isdir(d)]
    patients.sort()
    
    for p in patients:
        os.chdir(p)
        print('Reading data of patient', p)
        l=[]
        
        # for each patient specify the edf files and the seizure files
        edf = [f for f in os.listdir() if os.path.isfile(f) and f.endswith('edf')]
        edf.sort()
        seizures = [f for f in os.listdir() if os.path.isfile(f) and f.endswith('seizures')]
        seizures.sort()

        # edf files contain 23 columns/channels and a 24th column that indicates the seizure
        arr = np.zeros((1,24))
        for e in edf:
            sigbufs = readEdfFile(e)
            print('Reading data file', e, 'with', sigbufs.shape[0], 'records')
            sigbufs = np.append(sigbufs, np.zeros((sigbufs.shape[0],1)), axis=1)
            if seizures and seizures[0].startswith(e):
                (start, length) = get_seizure_period(seizures[0])
                print('Reading seizure file', seizures[0], ': (start =',start,'sec, length =',length,'sec) or', '(start =',start*sample_rate,'record, end =',(start+length)*sample_rate,'record)')
                for i in range(start*sample_rate, (start+length)*sample_rate+1): sigbufs[i][23] = 1
                seizures.pop(0)
            arr = np.concatenate([arr, sigbufs])
        arr = np.delete(arr, 22, axis=1)
        l.append(arr)
        os.chdir('..')
    os.chdir('..')
    
    del path, patients, p, edf, seizures, arr, e, sigbufs, start, length, i
    
    df = pd.DataFrame(np.concatenate(l), columns=['FP1-F7','F7-T7','T7-P7', 'P7-O1','FP1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 'FP2-F4','F4-C4', 'C4-P4', 'P4-O2', 'FP2-F8', 'F8-T8', 'T8-P8', 'P8-O2','FZ-CZ', 'CZ-PZ', 'P7-T7', 'T7-FT9', 'FT9-FT10', 'FT10-T8', 'seizure'])
    df.drop(0, inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

## Feature Extraction

### Left and Right Hemispheric Channel Information

In [ ]:
def leftRightHemisphericChannels(df):
    ndf = pd.DataFrame()
    ndf['AvgLeftHemisphere'] = (df['F3-C3']+df['C3-P3'])/2
    ndf['AvgRightHemisphere'] = (df['F4-C4']+df['C4-P4'])/2
    ndf['seizure'] = df['seizure']
    return ndf

### Time Domain Features

In [ ]:
'''
https://stackoverflow.com/questions/30272538/python-code-for-counting-number-of-zero-crossings-in-an-array
https://stackoverflow.com/questions/5613244/root-mean-square-in-numpy-and-complications-of-matrix-and-arrays-of-numpy
'''
def ComputeTimeDomainFeatures(signal):
    mean = np.mean(signal)
    var = np.var(signal)
    skew = stats.skew(signal)
    kurt = stats.kurtosis(signal)
    std = np.std(signal)
    median = np.median(signal)
    zcr = ((signal[:-1] * signal[1:]) < 0).sum()
    cv = stats.variation(signal)
    rms = np.sqrt(signal.dot(signal)/signal.size)
    p2p = signal.max() - signal.min()
    sampEn = entropy.sample_entropy(signal, 1)[0]
    return mean, var, skew, kurt, std, median, zcr, cv, rms, p2p, sampEn

### Frequency Features

In [ ]:
'''
Compute the average bandpower of an EEG signal
https://raphaelvallat.com/bandpower.html
'''
def psd(x, fs, win):
    bands = [0.5, 4, 8, 12, 30, 100]
    freqs, psd = signal.welch(x, fs, nperseg=win)
    avg_power=[]
    while len(bands)>1:
        idx = np.logical_and(freqs >= bands[0], freqs <= bands[1])
        power_simps = integrate.simps(psd[idx], dx=bands[1]-bands[0])
        avg_power.append(power_simps)
        bands=np.copy(bands[1:])
    for p in avg_power:
        yield p

### Correlation Features

In [ ]:
def ComputeCorrelation (left, right):
    return abs(np.correlate(left, right, 'full')).max()

In [ ]:
def featureExtraction (ndf, sample_rate, step):
    print('Computing Features...')
    ft = pd.DataFrame()
    c = 0
    for i in range (0, ndf.shape[0], step):
        print('Compute features for chunk from', i, 'to', i+step)
        temp = ndf.iloc[i:i+step]
        left = np.array(temp['AvgLeftHemisphere'])
        right = np.array(temp['AvgRightHemisphere'])

        # Time Domain Features
        ft.loc[c,'Lmean'], ft.loc[c,'Lvar'], ft.loc[c,'Lskew'],ft.loc[c,'Lkurt'], ft.loc[c,'Lstd'], ft.loc[c,'Lmedian'], ft.loc[c,'Lzcr'], ft.loc[c,'Lcv'], ft.loc[c,'Lrms'], ft.loc[c,'Lp2p'],ft.loc[c,'LsampEn'] = ComputeTimeDomainFeatures(left)
        ft.loc[c,'Rmean'], ft.loc[c,'Rvar'], ft.loc[c,'Rskew'],ft.loc[c,'Rkurt'], ft.loc[c,'Rstd'], ft.loc[c,'Rmedian'], ft.loc[c,'Rzcr'], ft.loc[c,'Rcv'], ft.loc[c,'Rrms'], ft.loc[c,'Rp2p'],ft.loc[c,'RsampEn'] = ComputeTimeDomainFeatures(right)

        # Frequency Domain Features
        ft.loc[c,'LdeltaPower'], ft.loc[c,'LthetaPower'], ft.loc[c,'LalphaPower'], ft.loc[c,'LbetaPower'], ft.loc[c,'LgammaPower'] = psd(left, sample_rate, 4*sample_rate)
        ft.loc[c,'RdeltaPower'], ft.loc[c,'RthetaPower'], ft.loc[c,'RalphaPower'], ft.loc[c,'RbetaPower'], ft.loc[c,'RgammaPower'] = psd(right, sample_rate, 4*sample_rate)

        
        # Correlation Features
        ft.loc[c, 'corr'] = ComputeCorrelation(left, right)

        ft.loc[c, 'seizure'] = temp['seizure'].value_counts().idxmax()
        c = c + 1
    print('Feature extraction completed')
    return ft

## Feature Normalization

In [ ]:
def featureNormalization(ft):
    return pd.DataFrame(preprocessing.scale(ft), columns=ft.columns)

## Feature Selection

In [ ]:
def dimentionalityReduction(features, pca_tolerance):
    pca = decomposition.PCA(n_components = pca_tolerance).fit(features)
    features = pd.DataFrame(pca.transform(features))
    return features

## Classification performance indexes 

In [ ]:
'''
TP : the number of segments that are correctly identified as ictal (x_true == x_pred == 1)
TN : the number of EEG segments that are correctly classified as non-ictal (x_true == x_pred == 0)
FP : the number of EEG segments that are incorrectly classified as ictal (x_true == 0 && x_pred == 1)
FN : the segments that are incorrectly classified as non-ictal (x_true == 1 && x_pred == 0)
'''
def metrics (true_arr, pred_arr):
    (tn, fp, fn, tp) = metr.confusion_matrix(true_arr, pred_arr).ravel()
    acc = metr.accuracy_score(true_arr, pred_arr)                  # Accuracy
    snv = tp/(tp + fn)                                             # Sensitivity or True Positive Rate (TPR)
    spc = tn/(tn + fp)                                             # Specificity or True Negative Rate (TNR)
    ppv = tp/(tp + fp)                                             # Precision or Positive Predictive Value (PPV)
    f1 = metrics.f1_score(true_arr, pred_arr)                      # F1 score
    mcc = metrics.matthews_corrcoef(true_arr, pred_arr)            # Matthews Correlation Coefficient
    kappa = metrics.cohen_kappa_score(true_arr, pred_arr)          # Cohen’s Kappa
    return acc, snv, spc, ppv, f1, mcc, kappa

## Main program

### Parameters

In [ ]:
sample_rate = 256
time_window = 5
step = time_window*sample_rate
pca_tolerance = 0.95
dataset = 'TestData' # 'CHB-MIT-Dataset'

### Seazure Detection Program

In [ ]:
df = read_and_store_data(dataset, sample_rate)
ndf = leftRightHemisphericChannels(df)
features = pd.DataFrame(featureExtraction (ndf, sample_rate, step))
features = featureNormalization(features)
features = dimentionalityReduction(features, pca_tolerance)